In [ ]:
import os
import sys
# Alter below path if needed to select custom casagui location
sys.path.insert(0, os.path.abspath('../../../casagui/'))
import casagui as casagui
print("Casagui location: " + casagui.__file__)
from casagui.apps import InteractiveClean
import asyncio

In [ ]:
import urllib,ssl, certifi, tarfile
ms_path = 'refim_point_withline.ms'
##
## where to fetch the demo measurement set
##
ms_url = "https://casa.nrao.edu/download/devel/casavis/data/refim_point_withline-ms.tar.gz"
##
## output image file name
##
img = 'test'

if not os.path.isdir(ms_path):
    try:
        context = ssl.create_default_context(cafile=certifi.where())
        tstream = urllib.request.urlopen(ms_url, context=context, timeout=400)
        tar = tarfile.open(fileobj=tstream, mode="r:gz")
        tar.extractall( )
    except urllib.error.URLError:
        print("Failed to open connection to "+ms_url)
        raise
        
if not os.path.isdir(ms_path):
    raise  RuntimeError("Failed to fetch measurement set")

In [ ]:
ic = InteractiveClean( vis=ms_path, imagename=img,
                       imsize=512,
                       cell='12.0arcsec',
                       specmode='cube',
                       interpolation='nearest',
                       nchan=5,
                       start='1.0GHz',
                       width='0.2GHz',
                       pblimit=-1e-05,
                       deconvolver='hogbom',
                       threshold='0.001Jy',
                       niter=2,
                       cycleniter=10,
                       cyclefactor=3,
                       scales=[0,3,10] )

In [ ]:
# Initialize pipes
ic.setup()

In [ ]:
def gen_port_fwd_cmd(ic):
    host = ic._pipe['control'].address[0]
    
    ports = [ic._pipe['control'].address[1],
            ic._pipe['converge'].address[1],
            ic._cube._pipe['image'].address[1],
            ic._cube._pipe['control'].address[1]]
    
    cmd = 'ssh'
    for port in ports:
        cmd += (' -L ' + str(port) + ':localhost:' + str(port))
    return cmd
    
print('Port Forwarding Command:\n')
gen_port_fwd_cmd(ic)

### Enter the above bash command to forward the necessary ports to the remote server

You will need to append the remote server's hostname to the end of this command.

In [ ]:
import bokeh.io
bokeh.plotting.reset_output()
bokeh.io.output_notebook()

## Start the Interactive Clean GUI

Ensure port forwarding was successful before running the below cell.

In [ ]:
# Start Interactive Clean GUI and display inline
async def process_events( ):
    async with ic.serve( ) as state:
        await state[0]

task_process=asyncio.create_task(process_events())